<a href="https://colab.research.google.com/github/Staures92/Projet-de-fin-Data-Afique-Hub/blob/main/Sentimentclient.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install streamlit

In [3]:
# Imoprtations des bibliothèques necessaires

import pandas as pd # pandas pour charger et explorer les données
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense
from keras.optimizers import Adam
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
import streamlit as st

# **Importation des données**

In [4]:
#Charger le fichier d'entraînement
df_train = pd.read_parquet('/content/drive/My Drive/train-00000-of-00001.parquet')

# Charger le fichier de test
df_test = pd.read_parquet('/content/drive/My Drive/test-00000-of-00001.parquet')

# df_train = pd.read_parquet('train-00000-of-00001.parquet')
# df_test = pd.read_parquet('test-00000-of-00001.parquet')

# **Exploration des données**

In [5]:
print("Train data:")
print(df_train.head(7))
print(df_train.info())

print("\nTest data:")
print(df_test.head(7))
print(df_test.info())

Train data:
   label                                               text
0      4  dr. goldberg offers everything i look for in a...
1      1  Unfortunately, the frustration of being Dr. Go...
2      3  Been going to Dr. Goldberg for over 10 years. ...
3      3  Got a letter in the mail last week that said D...
4      0  I don't know what Dr. Goldberg was like before...
5      4  Top notch doctor in a top notch practice. Can'...
6      4  Dr. Eric Goldberg is a fantastic doctor who ha...
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 650000 entries, 0 to 649999
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   label   650000 non-null  int64 
 1   text    650000 non-null  object
dtypes: int64(1), object(1)
memory usage: 9.9+ MB
None

Test data:
   label                                               text
0      0  I got 'new' tires from them and within two wee...
1      0  Don't waste your time.  We had two different p...
2  

**Observation** : Les données fournies présentent une structure standard avec deux colonnes, "label" et "text", et une répartition entre un ensemble d'entraînement (650 000 entrées) et un ensemble de test (50 000 entrées).
La colonne "label" est de type "int64", indiquant que les sentiments sont représentés par des nombres entiers.
La colonne "text" est de type "object", ce qui est typique pour stocker des chaînes de caractères dans pandas.

Cependant, les labels (étiquettes) indiquent une classification à 5 (de 0  à 4)classes, et les exemples de texte sont en anglais.

# **Prétraitement des données**

In [6]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')
stop_words = set(stopwords.words('english')) # utiliser les stop words anglais puisque les exemples de texte sont en anglais

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [7]:
 # Le prétraitement inclut la suppression de la ponctuation, URLs,
 # hashtags, mentions d'utlisateurs et  la conversion en minuscules,
 # la tokenization et la suppression des stop words

 # definir une fonction de prétraitement de texte
def preprocess_text(text):
    # Supprimer les caractères spéciaux
    text = re.sub(r'[^\w\s]', '', text)

    # Supprimer les URLs
    text = re.sub(r'http\S+', '', text)

    # Supprimer les hashtags (mots commençant par #)
    text = re.sub(r'#\w+', '', text)

    # Supprimer les mentions d'utilisateurs (mots commençant par @)
    text = re.sub(r'@\w+', '', text)

    # Convertir en minuscules
    text = text.lower()

    # Tokenization
    # Exemple d'utilisation de la fonction word_tokenize
    # supposons que text = "Check out this awesome #Python tutorial by @katwa! #programming https://www.example.com"
    # tokens va retourner un vecteur de mot individuel ['check', 'out', 'this', 'awesome', 'python', 'tutorial', 'by', 'katwa', 'programming']
    tokens = word_tokenize(text)

    # Supprimer les stop words (mots vides) du texte tokenisé. Exemple des mots vide dans le text ('out', 'this', 'by')
    # qui sont souvent supprimés lors du traitement de texte pour analyser le contenu principal
    tokens = [word for word in tokens if word not in stop_words] # Resultat: ['check', 'awesome', 'python', 'tutorial', 'katwa', 'programming']
    return ' '.join(tokens) # Retourne une chaîne unique en joignant tous les mots traités avec un espace (check awesome python tutorial katwa programming)

In [8]:
# Appliquer la fonction de prétraitement 'preprocess_text' à les colonnes'text' des deux DataFrames 'df_train' et 'df_test' et
# stocker le résultat dans une nouvelle colonne appelée 'processed_text'

df_train['processed_text'] = df_train['text'].apply(preprocess_text)
df_test['processed_text'] = df_test['text'].apply(preprocess_text)

In [9]:
df_train.head(3), df_test.head(3)

(   label                                               text  \
 0      4  dr. goldberg offers everything i look for in a...   
 1      1  Unfortunately, the frustration of being Dr. Go...   
 2      3  Been going to Dr. Goldberg for over 10 years. ...   
 
                                       processed_text  
 0  dr goldberg offers everything look general pra...  
 1  unfortunately frustration dr goldbergs patient...  
 2  going dr goldberg 10 years think one 1st patie...  ,
    label                                               text  \
 0      0  I got 'new' tires from them and within two wee...   
 1      0  Don't waste your time.  We had two different p...   
 2      0  All I can say is the worst! We were the only 2...   
 
                                       processed_text  
 0  got new tires within two weeks got flat took c...  
 1  dont waste time two different people come hous...  
 2  say worst 2 people place lunch place freezing ...  )

# **Entraînement et evaluations des modèles de classification des sentiments**

***1. Modèle RNN (LSTM)***


**Préparation des données pour l'entraînement du modèle**

In [10]:
# Initialize tokenizer
#tokenizer = Tokenizer()
#tokenizer.fit_on_texts(df_train['processed_text'])


In [11]:
# Convert text data to sequences
#X_train_sequences = tokenizer.texts_to_sequences(df_train['processed_text'])
#X_test_sequences = tokenizer.texts_to_sequences(df_test['processed_text'])

In [12]:
# Pad sequences to a fixed length
# The padding step ensures that all sequences have the same length, which is crucial for training deep learning models like LSTM
#max_length = 100
#X_train_padded = pad_sequences(X_train_sequences, maxlen=max_length, padding='post')
#X_test_padded = pad_sequences(X_test_sequences, maxlen=max_length, padding='post')



# Define the target variable
y_train = df_train['label'].values
y_test = df_test['label'].values

In [ ]:

X_train = df_train['processed_text'].values  # Convert to numpy array
X_test = df_test['processed_text'].values

# Ensure X_train & X_test are in a 1D array of strings
X_train= X_train.astype(str)
X_test = X_test.astype(str)

In [ ]:
# Create a pipeline with TF-IDF vectorizer and Logistic Regression
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=10000)),
    ('clf', LogisticRegression(C=1.0, max_iter=1000, n_jobs=-1))
])



In [ ]:
# Train the model
pipeline.fit(X_train, y_train)



In [ ]:
# Make predictions on the test set
y_pred = pipeline.predict(X_test)

# Print the classification report
print(classification_report(y_test, y_pred))

In [ ]:
# Save the model
import joblib
joblib.dump(pipeline, 'client_review_classifier.joblib')

In [ ]:
# Define the vocabulary size based on the tokenizer's word index
# vocab_size = total number of unique words or tokens in the tokenized dataset

vocab_size = len(tokenizer.word_index) + 1  # Adding 1 for the padding token

In [ ]:
# Define the embedding dimension

# The embedding dimension is a hyperparameter that determines the size of the vector representations of the words in the embedding layer
# the embedding dimension value is often chosen to be between 100 to 300 for medium to large datasets.
# Since we have a considerable number of samples in the training data, we can opt for a larger embedding dimension
# to capture more intricate relationships between words in the vocabulary

embedding_dim = 150  # In essence, this represents the dimensionality of the embedding space
